In [1]:
# Importing essential modules

import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.graph_objs import *
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
%matplotlib inline

In [2]:
# reading file

df = pd.read_excel("DIMEVENT - Useful.xlsx")

In [ ]:
# Making dashboard app for Keneddy space center where we will come to know from year 2011 to 2014 that which package has
# been opted most by the how many users and which package incurs more revenue among others.

app = dash.Dash("KSC")


# Populating layout with HTML and graph components
app.layout = html.Div([html.H2("ProductLOB"),
    html.Div([dcc.Dropdown(id="Product_LOB",
                options=[{
                    'label': i,
                    'value': i
                } for i in df['ProductLOB'].unique()],
                value='estab'),
              dcc.Graph(id='Bar-graph'),
    dcc.Slider(
        id='year-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].min(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
],style={'width': '95%','display': 'inline-block'})])


# Adding callbacks to support the interactive componets
@app.callback(
    dash.dependencies.Output('Bar-graph', 'figure'),
    [dash.dependencies.Input('year-slider', 'value'),
    dash.dependencies.Input('Product_LOB', 'value')])


def update_graph(year, product):
    
    df_ = df[['ProductLOB','ProductLOBSummary','ProductDescription','Year','EventDate','EventAmount','Quantity']]
    # df_ = df_[df_['Quantity'] >0]
    df_ = df_[(df_['Year'] == year)].reset_index(drop=True)
    table = df_.groupby(['ProductLOB', 'ProductDescription'], as_index=False).sum()

    data_amount = {}
    data_count = {}
    for e in table['ProductLOB'].unique():
        imd = table[table['ProductLOB']== e]
        amount = {}
        counts = {}
        for i,j,k in zip(imd['ProductDescription'],imd['EventAmount'], imd['Quantity']):
            if '(Taxable)' in i:
                i = i.split('(Taxable)')[0].strip()
            else:
                i = i.split('(Non-Taxable)')[0].strip()
            if i not in amount:
                amount[i] = []
                counts[i] = []
                amount[i].append(j)
                counts[i].append(k)
            else:
                amount[i].append(j)
                counts[i].append(k)
        data_amount[e] = amount
        data_count[e] = counts
    
    
    amt =[]
    for p in data_amount.keys():
        f = []
        for v in data_amount[p].values():
            f = f + v
        amt.append(sum(f))
    
    ct =[]
    for t in data_count.keys():
        q = []
        for r in data_count[t].values():
            q = q + r
        ct.append(sum(q))
    
    
    if (year in [2011, 2012, 2013, 2014]) and (product in list(table['ProductLOB'].unique())):
        amt_= [sum(i) for i in data_amount[product].values()]
        ct_ = [sum(i) for i in data_count[product].values()]
        txt_ = list(zip("Frequency :" + pd.Series(np.array(ct_).astype('str')), pd.Series(list(data_count[product].keys())) ))
        title_ = product + " ( Total amount : $" + format(sum(amt_),".2f")+ " | Total Quantities : " + str(sum(ct_))+ " )"
        trace = go.Bar(
        x= list(data_amount[product].keys()),
        y = amt_
        ,text = txt_,
        marker=dict(
            color= list(range(len(amt_)))))
        
    elif (year in [2011, 2012, 2013, 2014]):
        title_="|------ KSC ------> "+"( Total amount : $" + format(sum(amt),".2f")+" | Total Quantities : "+str(sum(ct))+" )"
        txt = list(zip("Frequency :" + pd.Series(np.array(ct).astype('str')), pd.Series(list(data_count.keys())) ))
        trace = go.Pie(
        labels= list(data_amount.keys()),
        values = amt
        ,text = txt,
        )
        
   
    
    return {
        'data': [trace],
        'layout':
        go.Layout(xaxis=dict(
        tickmode='linear',
        ticks='outside',
        tick0=0,
        dtick=0.25,
        ticklen=8,
        tickwidth=4,
        tickcolor='#000'),
                yaxis={'title': 'Revenue-------->'}, title= title_
            )
    }


if __name__ == '__main__':
    app.server.run(debug  = False)

 * Serving Flask app "KSC" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2020 01:33:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2020 01:33:39] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1561357055 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2020 01:33:39] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1561357054 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2020 01:33:39] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1561357059 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2020 01:33:39] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1561357054 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2020 01:33:39] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1561357065 HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2020 01:33:39] "GET /_dash-component-suites/dash_core_components/dash_core_components.min.js?v=1.0.0&m=1561357059 HTTP/1.1" 200 -
127.0.0.1 - - 